In [ ]:
# Instalación de dependencias
!py -m pip install -r ../requirements.txt

In [12]:
# Librerías básicas
import pandas as pd
import numpy as np
from random import randint


from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from mlxtend.frequent_patterns import apriori, association_rules

import re

from model import Data

In [13]:
# Cargado del dataset `Calidad_del_agua_del_Rio_Cauca_20240919`
REMOTE_ROUTE: str = "https://www.datos.gov.co/resource/d3ft-wu2b.csv"   # ! 2256 registros Y 56 columnas ! #
LIMIT = 1000  # Límite de filas por solicitud
OFFSET = 0  # Offset inicial
full_dataframe: pd.DataFrame = pd.DataFrame()

try:
    from google.colab import drive
    %matplotlib inline
    #drive.mount("/content/drive")

    while True:
        # Construimos la URL con los parámetros limit y offset
        query_url = f"{REMOTE_ROUTE}?$limit={LIMIT}&$offset={OFFSET}"
        # Descargamos los datos
        df_chunk = pd.read_csv(query_url)
        # Si no hay más datos, rompemos el bucle
        if df_chunk.empty:
            break
        # Concatenamos los datos descargados
        full_dataframe = pd.concat([full_dataframe, df_chunk], ignore_index=True)
        # Incrementamos el offset
        OFFSET += LIMIT

except ImportError as e:
    print(f"We are not in a Google Colab environment ({e}), we will use a local route.")
    try:
        # Si están precargados cargamos los datos directamente
        full_dataframe = pd.read_csv("./sample_data/local_data.csv")
    except FileNotFoundError:
        while True:
            # Construimos la URL con los parámetros limit y offset
            query_url = f"{REMOTE_ROUTE}?$limit={LIMIT}&$offset={OFFSET}"
            # Descargamos los datos
            df_chunk = pd.read_csv(query_url)
            # Si no hay más datos, rompemos el bucle
            if df_chunk.empty:
                break
            # Concatenamos los datos descargados
            full_dataframe = pd.concat([full_dataframe, df_chunk], ignore_index=True)
            # Incrementamos el offset
            OFFSET += LIMIT
            # Guardamos una copia local
            full_dataframe.to_csv("./sample_data/local_data.csv", index=False)

# Mostramos las primeras filas
full_dataframe.shape
full_dataframe.head()


We are not in a Google Colab environment (No module named 'google'), we will use a local route.


,fecha_de_muestreo,estaciones,ph,temperatura_c,color_upc,turbiedad_unt,solidos_totales_mg_sst_l,solidos_suspendidos_totales,solidos_disueltos_mg_sd_l,demanda_bioquimica_de_oxigeno,...,cromo_total_mg_cr_l,cromo_disuelto_mg_cr_l_,niquel_total_mg_ni_l,niquel_disuelto_mg_ni_l_,plomo_total_mg_pb_l,plomo_disuelto_mg_pb_l_,mercurio_g_hg_l_,coliformes_totales_nmp_100,coliformes_fecales_nmp_100,caudal_m3_s
0,1998-12-19T00:00:00.000,YOTOCO,7.1,4.1,NaN,4.1,NaN,110.0,NaN,4.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998-12-19T00:00:00.000,MEDIACANOA,7.0,2,NaN,2.0,NaN,130.0,NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998-12-19T00:00:00.000,PASO DE LA TORRE,7.0,22.9,NaN,3.4,NaN,153.3,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-05-09T00:00:00.000,ANTES SUAREZ,6.6,NaN,NaN,NaN,157.0,29.3,127.7,0.5,...,NaN,NaN,0,NaN,NaN,NaN,NaN,2.4*10E4,23,NaN
4,1990-01-10T00:00:00.000,ANTES RIO OVEJAS,6.7,NaN,NaN,NaN,143.0,65.0,78.0,2.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4*10E4,24*10E4,NaN


## Proceso ETL

Extracción, transformación y carga (Extract Transform Load, ETL) es el proceso que las organizaciones impulsadas por datos utilizan para recopilar datos de distintas fuentes para luego reunirlos a fin de facilitar el descubrimiento, la generación de informes, el análisis y la toma de decisiones.

In [14]:
# Función para limpiar y convertir a numérico los valores de las columnas
def clean_and_convert(value):
    if pd.isna(value):
        return np.nan

    # Remover símbolos indeseados como '>', '<', '*', y cambiar comas por puntos
    value = str(value).replace('>', '').replace('<', '').replace('*', '').replace(',', '.')

    # Usar regex para identificar y manejar exponentes (como E, 10E)
    if re.search(r'[eE]', value):
        try:
            return float(value)
        except ValueError:
            return np.nan

    # Usar regex para manejar notación científica con el formato 'x*10^y'
    if re.search(r'\d+\.\d*\*10E[\+\-]?\d+', value):
        try:
            base, exponent = value.split('*10E')
            return float(base) * 10 ** float(exponent)
        except ValueError:
            return np.nan

    # Intentar convertir el valor directamente a float
    try:
        return float(value)
    except ValueError:
        return np.nan

In [21]:
# Seleccionamos todas las columnas numéricas
NUM_COLS = full_dataframe.iloc[:, 2:].columns

# Seleccionamos todas las columnas numéricas y las limpiamos
numeric: pd.DataFrame = full_dataframe[NUM_COLS].applymap(clean_and_convert)


# Eliminación de columnas con más del 50% de valores nulos y tratamiento de valores nulos
threshold = 0.5
numeric = numeric.loc[:, numeric.isnull().mean() < threshold]
numeric = numeric.fillna(numeric.mean())


full_dataframe = pd.concat([full_dataframe.iloc[:,:2], numeric], axis=1)
full_dataframe.describe()

C:\Users\overd\AppData\Local\Temp\ipykernel_12500\143399176.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  numeric: pd.DataFrame = full_dataframe[NUM_COLS].applymap(clean_and_convert)


,ph,temperatura_c,color_upc,turbiedad_unt,solidos_totales_mg_sst_l,solidos_suspendidos_totales,solidos_disueltos_mg_sd_l,demanda_bioquimica_de_oxigeno,demanda_quimica_de_oxigeno,oxigeno_disuelto_mg_o2_l,...,cloruros_mg_cl_l,fosforo_total_mg_p_l,fosfatos_mg_po4_l,sulfatos_mg_so4_l,cadmio_total_mg_cd_l,cromo_total_mg_cr_l,niquel_total_mg_ni_l,plomo_total_mg_pb_l,coliformes_totales_nmp_100,coliformes_fecales_nmp_100
count,2254.000000,2254.000000,2254.000000,2254.000000,2254.000000,2254.000000,2254.00000,2254.000000,2254.000000,2254.000000,...,2254.000000,2254.000000,2254.000000,2254.000000,2254.000000,2254.000000,2254.000000,2254.000000,2.254000e+03,2.254000e+03
mean,7.044717,22.022321,135.280069,134.044514,275.185570,161.363904,117.46440,5.006724,28.351104,4.061527,...,7.027014,0.268925,0.094475,18.368555,0.205591,0.165057,0.075543,0.148743,1.240005e+11,1.180217e+13
std,0.402512,3.355730,223.501644,172.613723,224.572612,201.345737,62.76937,15.676351,36.120875,2.938073,...,19.809453,0.804148,0.441219,16.436793,1.672273,0.112031,0.173481,0.158637,5.076992e+12,5.055274e+14
min,4.100000,0.000000,0.000000,1.000000,0.000000,1.200000,0.00000,0.100000,1.460000,0.000000,...,0.112000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,6.800000,21.000000,43.725000,32.000000,143.250000,39.000000,84.00000,1.940000,12.500000,2.320000,...,4.092500,0.093175,0.056200,14.200000,0.010000,0.043000,0.017725,0.060000,2.400000e+04,9.300000e+03
50%,7.060000,22.022321,80.750000,74.000000,209.000000,90.000000,109.00000,3.000000,19.700000,3.865000,...,5.970000,0.172000,0.064000,18.368555,0.040000,0.200000,0.090000,0.060000,2.400000e+05,9.300000e+04
75%,7.270000,24.000000,135.280069,147.000000,319.000000,210.000000,135.87500,4.747500,30.545000,5.600000,...,8.020000,0.268925,0.100000,20.700000,0.040000,0.288000,0.090000,0.310000,1.500000e+07,2.400000e+06
max,9.700000,32.700000,2956.000000,1900.000000,2361.000000,2112.000000,864.40000,427.000000,706.000000,51.500000,...,738.000000,14.180000,20.100000,696.800000,24.400000,0.997000,7.700000,1.540000,2.410000e+14,2.400000e+16


In [22]:
# ** Conversión de la columna de fechas (FECHA_DE_MUESTREO) **
full_dataframe[Data.FECHA_DE_MUESTREO] = pd.to_datetime(
    full_dataframe[Data.FECHA_DE_MUESTREO],
    errors="coerce",  # Convierte fechas inválidas a NaT
    infer_datetime_format=True,  # Infiere formatos mixtos
)

# ** Función para generar una fecha aleatoria en un año reciente (2022 o 2023) **
def generar_fecha_random():
    year = np.random.choice([2022, 2023])
    month = randint(1, 12)
    day = randint(1, 28)  # Evitar problemas con días fuera de rango
    hour = randint(0, 23)
    minute = randint(0, 59)
    return pd.Timestamp(year=year, month=month, day=day, hour=hour, minute=minute)

# ** Llenar las fechas 'NaT' con una fecha reciente aleatoria **
full_dataframe[Data.FECHA_DE_MUESTREO] = full_dataframe[Data.FECHA_DE_MUESTREO].apply(
    lambda x: generar_fecha_random() if pd.isna(x) else x
)

# ** Ordenar el DataFrame por la columna de fechas **
full_dataframe = full_dataframe.sort_values(by=Data.FECHA_DE_MUESTREO)

# Verificar la distribución después del reemplazo
print(full_dataframe[Data.FECHA_DE_MUESTREO].value_counts())

fecha_de_muestreo
2015-03-18 00:00:00    19
2017-05-09 00:00:00    19
2019-08-14 00:00:00    19
2019-06-18 00:00:00    19
2018-08-14 00:00:00    19
                       ..
2023-06-21 15:30:00     1
2023-06-21 17:30:00     1
2023-07-28 05:45:00     1
2023-08-27 11:03:00     1
2023-09-18 00:04:00     1
Name: count, Length: 210, dtype: int64


C:\Users\overd\AppData\Local\Temp\ipykernel_12500\1818644971.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  full_dataframe[Data.FECHA_DE_MUESTREO] = pd.to_datetime(


In [16]:
# ** Imputar la columna 'estaciones' **

# Calcular la moda (valor más frecuente) en la columna 'estaciones'
moda_estaciones = full_dataframe['estaciones'].mode()[0]

# Rellenar los valores NaN en 'estaciones' con la moda
full_dataframe['estaciones'].fillna(moda_estaciones, inplace=True)

# Verificar que ya no haya valores NaN en la columna 'estaciones'
print(full_dataframe['estaciones'].isna().sum())

0


C:\Users\overd\AppData\Local\Temp\ipykernel_12500\2772320851.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_dataframe['estaciones'].fillna(moda_estaciones, inplace=True)


# VARIABLE CATEGÓRICA

In [23]:
# Definir los metales relevantes
TARGET = Data.CONTAMINACION_METALICA
metales = [
    Data.HIERRO_TOTAL_MG_FE_L,
    Data.MANGANESO_TOTAL_MG_MN_L,
    Data.SODIO_TOTAL_MG_NA_L,
    Data.POTASIO_TOTAL_MG_K_L,
    Data.PLOMO_TOTAL_MG_PB_L,
    Data.CROMO_TOTAL_MG_CR_L,
    Data.CADMIO_TOTAL_MG_CD_L,
    Data.NIQUEL_TOTAL_MG_NI_L,
    Data.COBRE_TOTAL_MG_CU_L,
    Data.ZINC_TOTAL_MG_ZN_L,
]
numeric = full_dataframe.iloc[:, 2:]

# Filtrar las columnas que existen en tu DataFrame
metales_presentes = [metal for metal in metales if metal in numeric.columns]

# Calcular los percentiles para cada metal
percentiles = {}
for metal in metales_presentes:
    percentiles[metal] = {
        "25": numeric[metal].quantile(0.25),
        "50": numeric[metal].quantile(0.50),
        "75": numeric[metal].quantile(0.75),
        "90": numeric[metal].quantile(0.90),
    }


# Función para asignar puntuación a cada metal en una muestra
def puntuacion_metal(concentracion, metal):
    if np.isnan(concentracion):
        return 0  # O decide cómo manejar NaN
    if concentracion <= percentiles[metal]["25"]:
        return 1
    elif concentracion <= percentiles[metal]["50"]:
        return 2
    elif concentracion <= percentiles[metal]["75"]:
        return 3
    elif concentracion <= percentiles[metal]["90"]:
        return 4
    else:
        return 5


# Función para calcular la puntuación total y asignar categoría
def contaminacion_metales(row):
    total_puntuacion = 0
    for metal in metales_presentes:
        concentracion = row[metal]
        puntuacion = puntuacion_metal(concentracion, metal)
        total_puntuacion += puntuacion

    # Ajustar los valores de N1, N2, N3 y N4 según tus datos
    N1 = len(metales_presentes) * 1.5  # Por ejemplo
    N2 = len(metales_presentes) * 2.5
    N3 = len(metales_presentes) * 3.5
    N4 = len(metales_presentes) * 4.5

    if total_puntuacion <= N1:
        return "viable"
    elif total_puntuacion <= N2:
        return "bajo"
    elif total_puntuacion <= N3:
        return "medio"
    elif total_puntuacion <= N4:
        return "alto"
    else:
        return "inviable"

# ** Parte 1: Calcular la columna de contaminación metálica **

# Aplicar la función 'contaminacion_metales' al DataFrame
full_dataframe['contaminacion_metalica'] = numeric.apply(contaminacion_metales, axis=1)
numeric['contaminacion_metalica'] = numeric.apply(contaminacion_metales, axis=1)
# Mostrar un resumen de la clasificación
print(full_dataframe['contaminacion_metalica'].value_counts())

# ** Almacenar el DataFrame con la data procesada en 'clean_data.csv' **
full_dataframe.to_csv("./sample_data/clean_data.csv", index=False)


contaminacion_metalica
bajo      1159
medio      747
viable     287
alto        61
Name: count, dtype: int64


Se encuentra cómo estos agentes metálicos contaminantes afectan a la calidad agua en Cauca, Colombia. Se tiene un conjunto de datos que contiene la concentración de los agentes metálicos contaminantes.

Enlace referente [INCA-2021](https://www.minvivienda.gov.co/sites/default/files/documentos/informe-nacional-de-calidad-del-agua-para-consumo-humano-inca-2021.pdf#page=22&zoom=100,0,0)